In [32]:
##
path = '/Users/jeebanswain/Documents/Kaggle Challenges/RossMan Store Sales/'
import os ## import OS library
os.chdir(path) ## Set the working directory
os.listdir(path) ## see the files in a given path

['.DS_Store',
 'sample_submission-2.csv',
 'store.csv',
 'test-3.csv',
 'train-3.csv']

In [33]:
## Import all required libraries
import pandas as pd
import numpy as np

In [34]:
train = pd.read_csv('train-3.csv') ## read training set 
store = pd.read_csv('store.csv') ## read store lavel details 
test = pd.read_csv('test-3.csv') ## read test set

In [35]:
### See the dimension of the all three files
print(train.shape)
print(test.shape)
print(store.shape)

(1017209, 9)
(41088, 8)
(1115, 10)


In [36]:
## see all the data set
train.head()## 

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [37]:
test.head() ## sales variable is to be predicted 

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1,1,0,0
1,2,3,4,2015-09-17,1,1,0,0
2,3,7,4,2015-09-17,1,1,0,0
3,4,8,4,2015-09-17,1,1,0,0
4,5,9,4,2015-09-17,1,1,0,0


In [38]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270,9,2008,0,NaN,NaN,NaN
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct"
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct"
3,4,c,c,620,9,2009,0,NaN,NaN,NaN
4,5,a,a,29910,4,2015,0,NaN,NaN,NaN


#### Understand The meaning of each Fields 
You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

##### Files

- train.csv - historical data including Sales
- test.csv - historical data excluding Sales
- sample_submission.csv - a sample submission file in the correct format
- store.csv - supplemental information about the stores

##### Data fields
Most of the fields are self-explanatory. The following are descriptions for those that aren't.

- Id - an Id that represents a (Store, Date) duple within the test set
- Store - a unique Id for each store
- Sales - the turnover for any given day (this is what you are predicting)
- Customers - the number of customers on a given day
- Open - an indicator for whether the store was open: 0 = closed, 1 = open
- StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
- SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
- StoreType - differentiates between 4 different store models: a, b, c, d
- Assortment - describes an assortment level: a = basic, b = extra, c = extended
- CompetitionDistance - distance in meters to the nearest competitor store
- CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
- Promo - indicates whether a store is running a promo on that day
- Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
- Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
- PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

#### Attributes can be used for predictions are 
- Dayofweek
- Open
- Promo
- StateHoliday
- SchoolHoliday

##### Lets create 1st Linear Regression model using above features for sales prediction

In [39]:
train.StateHoliday = train.StateHoliday.astype('str') ## Converting the StateHoliday to string type
train = pd.get_dummies(data=train,columns=['StateHoliday'])
train.columns

Index([u'Store', u'DayOfWeek', u'Date', u'Sales', u'Customers', u'Open',
       u'Promo', u'SchoolHoliday', u'StateHoliday_0', u'StateHoliday_a',
       u'StateHoliday_b', u'StateHoliday_c'],
      dtype='object')

### Build the Model with 3 variables 'Open','Promo','SchoolHoliday'

In [10]:


from sklearn.linear_model import LinearRegression ## Import the linear Regression from sklearn
##Define X and Y
y_train = train.Sales ## y is sales here which we want to predict
X_train = train[['Open','Promo','SchoolHoliday']] ## Select X == features to be used

lm = LinearRegression() ## Define the linear model
lm.fit(X=X_train,y=y_train)

# predict the Sales for training set
predicted_sales = lm.predict(X_train) ### use same features ie train_x for prediction
train['predicted_sales'] = predicted_sales ## assign the predicted sales train set for actual tp predicted sales comp
train['percentage_error'] = abs(train.Sales - train.predicted_sales)/(train.Sales)## Chek the % Error
print(train.percentage_error.mean()) ## It will return inf -- which since base value is 0 
## Lets calculates the error for non zero sales 
print(train[train.Sales>0].percentage_error.mean()) ## 35% is the prediction error

/Users/jeebanswain/anaconda2/lib/python2.7/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

### Build the Model with 3 variables 'Open','Promo','SchoolHoliday' and Stateholiday dummy variables

In [54]:
from sklearn.linear_model import LinearRegression ## Import the linear Regression from sklearn
##Define X and Y
y_train = train.Sales ## y is sales here which we want to predict
X_train = train[['Open','Promo','SchoolHoliday','StateHoliday_0', 'StateHoliday_a']] ## Select X == features to be used

lm = LinearRegression() ## Define the linear model
lm.fit(X=X_train,y=y_train)

# predict the Sales for training set
predicted_sales = lm.predict(X_train) ### use same features ie train_x for prediction
train['predicted_sales'] = predicted_sales ## assign the predicted sales train set for actual tp predicted sales comp
train['percentage_error'] = abs(train.Sales - train.predicted_sales)/(train.Sales)## Chek the % Error
print(train.percentage_error.mean()) ## It will return inf -- which since base value is 0 
## Lets calculates the error for non zero sales 
print(train[train.Sales>0].percentage_error.mean()) ## 35% is the prediction error

inf
0.351301988195


In [69]:
#

inf
0.351607936744
